This document is following the SQLAlchemy 1.4 Document Object Relational Tutorial  
https://docs.sqlalchemy.org/en/14/orm/tutorial.html#version-check


# Settings

In [145]:
# Connecting

from sqlalchemy import create_engine
engine = create_engine('sqlite:///orm_test4.db', echo=True)

In [146]:
# Declare a Mapping

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [147]:
# Create a Schema

from sqlalchemy import Column, Integer, String, DateTime
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    date = Column(DateTime)
    def __repr__(self):
       return "<User(name='%s', fullname='%s', nickname='%s',date='%s')>" % (
                            self.name, self.fullname, self.nickname, self.date)

In [148]:
from sqlalchemy import Sequence
Column(Integer, Sequence('user_id_seq'), primary_key=True)

Column(None, Integer(), table=None, primary_key=True, nullable=False, default=Sequence('user_id_seq'))

In [149]:
Base.metadata.create_all(engine)

2022-07-05 11:39:56,536 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-05 11:39:56,537 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-07-05 11:39:56,537 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-05 11:39:56,538 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-07-05 11:39:56,539 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-05 11:39:56,541 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	date DATETIME, 
	PRIMARY KEY (id)
)


2022-07-05 11:39:56,542 INFO sqlalchemy.engine.Engine [no key 0.00090s] ()
2022-07-05 11:39:56,571 INFO sqlalchemy.engine.Engine COMMIT


In [150]:
from datetime import datetime
now = datetime.now()

In [151]:
# Create an instance of the mapped Class

ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname', date=now)
print(ed_user)
print(ed_user.name)
print(ed_user.fullname)
print(ed_user.nickname)
print(str(ed_user.id))

<User(name='ed', fullname='Ed Jones', nickname='edsnickname',date='2022-07-05 11:39:56.675644')>
ed
Ed Jones
edsnickname
None


# Creating a Session

In [152]:
from sqlalchemy.orm import sessionmaker

session_maker = sessionmaker(bind=engine)
session = session_maker()

In [153]:
session.add(ed_user)
print(ed_user)
ed_user in session.dirty

<User(name='ed', fullname='Ed Jones', nickname='edsnickname',date='2022-07-05 11:39:56.675644')>


False

In [154]:
session.commit()

2022-07-05 11:39:56,940 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-05 11:39:56,942 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname, date) VALUES (?, ?, ?, ?)
2022-07-05 11:39:56,942 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ('ed', 'Ed Jones', 'edsnickname', '2022-07-05 11:39:56.675644')
2022-07-05 11:39:56,949 INFO sqlalchemy.engine.Engine COMMIT


In [155]:
# https://docs.sqlalchemy.org/en/14/orm/session_api.html#session-and-sessionmaker

# Adding data

In [156]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy',date=now),
    User(name='mary', fullname='Mary Contrary', nickname='mary', date=now),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy', date=now)])

session.commit()

2022-07-05 11:39:57,070 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-05 11:39:57,071 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname, date) VALUES (?, ?, ?, ?)
2022-07-05 11:39:57,071 INFO sqlalchemy.engine.Engine [cached since 0.1301s ago] ('wendy', 'Wendy Williams', 'windy', '2022-07-05 11:39:56.675644')
2022-07-05 11:39:57,073 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname, date) VALUES (?, ?, ?, ?)
2022-07-05 11:39:57,075 INFO sqlalchemy.engine.Engine [cached since 0.1329s ago] ('mary', 'Mary Contrary', 'mary', '2022-07-05 11:39:56.675644')
2022-07-05 11:39:57,075 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname, date) VALUES (?, ?, ?, ?)
2022-07-05 11:39:57,076 INFO sqlalchemy.engine.Engine [cached since 0.1345s ago] ('fred', 'Fred Flintstone', 'freddy', '2022-07-05 11:39:56.675644')
2022-07-05 11:39:57,076 INFO sqlalchemy.engine.Engine COMMIT


In [157]:
session.new

IdentitySet([])

# Query  

In [158]:
from sqlalchemy import select

select vs execute(select)

In [159]:
query_all = session.query(User)
print(query_all)
print(type(query_all))

SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, users.date AS users_date 
FROM users
<class 'sqlalchemy.orm.query.Query'>


In [160]:
query_exe = session.execute(select(User))
print(query_exe)
print(type(query_exe))

2022-07-05 11:39:57,334 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-05 11:39:57,335 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, users.nickname, users.date 
FROM users
2022-07-05 11:39:57,335 INFO sqlalchemy.engine.Engine [generated in 0.00040s] ()
<class 'sqlalchemy.engine.result.ChunkedIteratorResult'>


In [161]:
for i in query_all:
    print(i)

2022-07-05 11:39:57,397 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, users.date AS users_date 
FROM users
2022-07-05 11:39:57,398 INFO sqlalchemy.engine.Engine [generated in 0.00054s] ()
<User(name='ed', fullname='Ed Jones', nickname='edsnickname',date='2022-07-05 11:39:56.675644')>
<User(name='wendy', fullname='Wendy Williams', nickname='windy',date='2022-07-05 11:39:56.675644')>
<User(name='mary', fullname='Mary Contrary', nickname='mary',date='2022-07-05 11:39:56.675644')>
<User(name='fred', fullname='Fred Flintstone', nickname='freddy',date='2022-07-05 11:39:56.675644')>


In [162]:
for i in query_exe:
    print(i)

(<User(name='ed', fullname='Ed Jones', nickname='edsnickname',date='2022-07-05 11:39:56.675644')>,)
(<User(name='wendy', fullname='Wendy Williams', nickname='windy',date='2022-07-05 11:39:56.675644')>,)
(<User(name='mary', fullname='Mary Contrary', nickname='mary',date='2022-07-05 11:39:56.675644')>,)
(<User(name='fred', fullname='Fred Flintstone', nickname='freddy',date='2022-07-05 11:39:56.675644')>,)


In [163]:
session.close()

2022-07-05 11:39:57,517 INFO sqlalchemy.engine.Engine ROLLBACK


In [164]:
for instance in session.query(User):
    print(instance)

2022-07-05 11:39:57,572 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-05 11:39:57,573 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, users.date AS users_date 
FROM users
2022-07-05 11:39:57,573 INFO sqlalchemy.engine.Engine [cached since 0.176s ago] ()
<User(name='ed', fullname='Ed Jones', nickname='edsnickname',date='2022-07-05 11:39:56.675644')>
<User(name='wendy', fullname='Wendy Williams', nickname='windy',date='2022-07-05 11:39:56.675644')>
<User(name='mary', fullname='Mary Contrary', nickname='mary',date='2022-07-05 11:39:56.675644')>
<User(name='fred', fullname='Fred Flintstone', nickname='freddy',date='2022-07-05 11:39:56.675644')>


In [165]:
for instance in session.execute(select(User)):
    print(instance)

2022-07-05 11:39:57,635 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, users.nickname, users.date 
FROM users
2022-07-05 11:39:57,636 INFO sqlalchemy.engine.Engine [cached since 0.3009s ago] ()
(<User(name='ed', fullname='Ed Jones', nickname='edsnickname',date='2022-07-05 11:39:56.675644')>,)
(<User(name='wendy', fullname='Wendy Williams', nickname='windy',date='2022-07-05 11:39:56.675644')>,)
(<User(name='mary', fullname='Mary Contrary', nickname='mary',date='2022-07-05 11:39:56.675644')>,)
(<User(name='fred', fullname='Fred Flintstone', nickname='freddy',date='2022-07-05 11:39:56.675644')>,)


In [166]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2022-07-05 11:39:57,694 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, users.date AS users_date 
FROM users ORDER BY users.id
2022-07-05 11:39:57,695 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [167]:
for instance in session.query(User).order_by(User.id)[1:3]:
    print(instance.name, instance.fullname)

2022-07-05 11:39:57,754 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, users.date AS users_date 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2022-07-05 11:39:57,755 INFO sqlalchemy.engine.Engine [generated in 0.00084s] (2, 1)
wendy Wendy Williams
mary Mary Contrary


In [176]:
for name in session.query(User).filter(User.name=='mary'):
    print(name.name)

2022-07-05 12:06:53,731 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, users.date AS users_date 
FROM users 
WHERE users.name = ?
2022-07-05 12:06:53,732 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ('mary',)
mary


In [168]:
for row in session.query(User.name.label('name_label')).all():
   print(row.name_label)

2022-07-05 11:39:57,819 INFO sqlalchemy.engine.Engine SELECT users.name AS name_label 
FROM users
2022-07-05 11:39:57,819 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
ed
wendy
mary
fred


In [177]:
# query.filter(User.name=='ed')
# query.filter(User.name != 'ed')
# query.filter(User.name.like('%ed%'))
# query.filter(User.name.in_(['ed','wendy','jack']))

In [ ]:
# from sqlalchemy import tuple_
# session.query.filter()

Return List vs Scala  
returns list .all() 
return a scalar .first()  

# End

In [169]:
session.close()

2022-07-05 11:39:57,878 INFO sqlalchemy.engine.Engine ROLLBACK
